## Importing modules and packages

In [11]:
# Importing packages
import osmnx  as ox
import pandas as pd
import numpy  as np
import folium
import os.path
import shapely
import geopandas as gpd
from IPython.display import IFrame

# Importing modules
import gr_mapmatch # Contains functions that perform the map matching of roads
# import gr_placematch # Contains functions that perform the map matching of places
import gr_utils # Contains useful geometry functions
import gr_plot # Contains plotting routines
# import gr_process
import roadmatch

# Configuring modules & packages
ox.settings.useful_tags_way = [
    "bridge","tunnel","name","highway","area","landuse","surface","tracktype"
] # Configuring which parameters we want to obtain from OSM

## Input parameters

In [2]:
trailname = 'gr122' # Name of the hiking trail to be considered (will search for trail.csv or trail.gpx as sources)

# Settings for trail2roads
points_per_batch = 100 # Subdivide the trail into batches of this many points
delta_roads = 0.010 # Tolerance around bounding box per trail section [deg]

# Settings for roads2places
points_per_batch_places = 100 # Subdivide the trail into batches of this many segments
delta_places = 0.015 # bbox delta in deg
buffersize = 0.00015 # buffer used when merging landuse places
tol_area = 15.0e-6 # buffer used to define which merged landuse places we consider

# Settings for places2processed
tol_d        = 0.75 # Consider a segment developed if it lies closer than tol_d to a developed area
tracktype_p0 = ['grade4','grade5']
tracktype_p1 = ['grade2','grade3']
tracktype_p2 = ['grade1']
surface_p0   = ['ground','grass','dirt','sand','earth','mud']
surface_p1   = ['unpaved','gravel','fine_gravel','wood','compacted','rocks','pebblestone','woodchips','snow','ice','salt']
highway_p1   = ['track','path','footway','bridleway']
types_slow   = ['pedestrian','track','footway','bridleway','steps','corridor','path']
types_heavy  = ['motorway','trunk','primary','secondary','tertiary']

## Loading GPX file

In [3]:
gpx = gr_utils.get_gpx(trailname)
gpx_coords = gr_mapmatch.trail_to_coords(gpx) # Convert the points into a list of [lat, lon] pairs

Loading trail points from <data_input/gr122.gpx>...
Finished loading.


## Matching GPX points to OSM nodes

In [4]:
# Generate the nodes dataframe [point_x, point_y, node_id, node_x, node_y, d2node]
# with point = a GPX point and node = the corresponding node
# and d2node = the distance from the GPX point to its corresponding node
filename_nodes = 'cache/' + trailname + '_nodes.csv'
if not os.path.isfile(filename_nodes): # The nodes file does not exist, construct it
    nodes = roadmatch.gpx_to_nodes(gpx, gpx_coords, points_per_batch, delta_roads)
    nodes.to_csv(filename_nodes)
else: # The nodes file exists, load it
    nodes = pd.read_csv(filename_nodes,index_col=0,
                            dtype={'point_x':float, 'point_y':float,
                                   'node_id':int,
                                   'node_x':float, 'node_y':float,
                                   'd2node':float})

In [5]:
nodes

,point_x,point_y,node_id,node_x,node_y,d2node
0,4.05118,51.27809,1649282241,4.051181,51.278079,0.000011
1,4.05111,51.27828,42291968,4.051096,51.278274,0.000015
2,4.05092,51.27827,393406001,4.050765,51.278272,0.000155
3,4.05025,51.27826,1871685302,4.050218,51.278269,0.000034
4,4.04961,51.27826,1871685302,4.050218,51.278269,0.000608
...,...,...,...,...,...,...
3943,3.51948,50.76068,4902407176,3.519843,50.761135,0.000582
3944,3.51984,50.76113,4902407176,3.519843,50.761135,0.000006
3945,3.51998,50.76117,4902407176,3.519843,50.761135,0.000142
3946,3.52009,50.76124,4902407176,3.519843,50.761135,0.000269


In [6]:
# Generate the pieces dataframe [node0, node1, gpx0, gpx1]
# it cuts up the GPX trail into pieces on which individual pathfinding can be done
filename_pieces = 'cache/' + trailname + '_pieces.csv'
if not os.path.isfile(filename_pieces): # The pieces file does not exist, construct it
    pieces = roadmatch.nodes_to_pieces(nodes)
    pieces.to_csv(filename_pieces)
else:
    pieces = pd.read_csv(filename_pieces,index_col=0,
                         dtype={'node0':int,'node1':int,
                                'gpx0':int,'gpx1':int})

In [7]:
pieces

,node0,node1,gpx0,gpx1
0,1649282241,42291968,0,1
1,42291968,393406001,1,2
2,393406001,1871685302,2,3
3,1871685302,393406011,3,12
4,393406011,393406780,12,15
...,...,...,...,...
791,4278294677,4278203763,3917,3926
792,4278203763,1287813114,3926,3932
793,1287813114,5339445555,3932,3936
794,5339445555,5339445593,3936,3938


In [8]:
npaths = 5

# Generate the segments dataframe [x0,y0,x1,y1,d_cart,d_osm,highway,surface,tracktype]
filename_segments = 'cache/' + trailname + '_segments.csv'
if not os.path.isfile(filename_segments): # The pieces file does not exist, construct it
    segments = roadmatch.pieces_to_segments(gpx,nodes,points_per_batch,delta_roads,pieces,npaths)
    segments.to_csv(filename_segments)
else:
    segments = pd.read_csv(filename_segments,index_col=0,
                         dtype={'x0':float,'y0':float,
                                'x1':float,'y1':float,
                               'd_cart':float,'d_osm':float,
                                'highway':str,'surface':str,'tracktype':str})

/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/osmnx/graph.py:488: UserWarning: the graph-level street_count attribute will likely be inaccurate when you set clean_periphery=False
  warnings.warn(msg)


Handling piece 38 of 796, spanning GPX points 101 through 102


/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/osmnx/graph.py:488: UserWarning: the graph-level street_count attribute will likely be inaccurate when you set clean_periphery=False
  warnings.warn(msg)


Handling piece 53 of 796, spanning GPX points 217 through 229


/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/osmnx/graph.py:488: UserWarning: the graph-level street_count attribute will likely be inaccurate when you set clean_periphery=False
  warnings.warn(msg)


...

/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/osmnx/graph.py:488: UserWarning: the graph-level street_count attribute will likely be inaccurate when you set clean_periphery=False
  warnings.warn(msg)


Handling piece 778 of 796, spanning GPX points 3810 through 3817


/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/osmnx/graph.py:488: UserWarning: the graph-level street_count attribute will likely be inaccurate when you set clean_periphery=False
  warnings.warn(msg)


Handling piece 791 of 796, spanning GPX points 3917 through 3926


/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/osmnx/graph.py:488: UserWarning: the graph-level street_count attribute will likely be inaccurate when you set clean_periphery=False
  warnings.warn(msg)


## Plot the matched route

In [15]:
matchedcoords = gr_plot.get_fullcoords_from_frame(segments)

# Map setup
mid = int(len(gpx_coords)/2)
chart = folium.Map(location=gpx_coords[mid], zoom_start=10, tiles="OpenStreetMap")

# Draw the GPX route
newline = folium.PolyLine(locations=gpx_coords, weight=2, color='red')
newline.add_to(chart)

# Draw the matched route
newline = folium.PolyLine(locations=matchedcoords, weight=2, color='black')
newline.add_to(chart)

# Render the map
filepath = f"cache/temp11.html"
chart.save(filepath)
IFrame(filepath, width=1000, height=500)

## Gathering ROAD data from OSM network

In [4]:
# filename_roads = 'cache/' + trailname + '_roads.csv'

# if not os.path.isfile(filename_roads): # The merged ROADS file does not exist, construct it
    
#     print('Merged ROADS file was not found, merging and saving...')
#     gr_mapmatch.trail2roads(trailname, trail, points_per_batch, delta) # Main batch processor
#     data_roads_raw = gr_utils.merge_roads(trailname, trail, points_per_batch) # Merge the different sections
#     data_roads = gr_mapmatch.remove_repeat_segments(data_roads_raw) # Remove backtracked sections
#     gr_utils.write_roads(trailname, data_roads) # Write the merged ROADS data
# #     print('Saved.')

# else: # The merged ROADS file does exist, just load it
    
#     print('Loading merged ROADS file...')
#     data_roads = gr_utils.read_roads(trailname) # Read the merged sections
#     print('Loaded.')

Loading merged ROADS file...
Loaded.


## Gathering PLACE data from OSM network

In [5]:
# filename_places = 'cache/' + trailname + '_places.csv'

# if not os.path.isfile(filename_places): # The merged PLACES file does not exist
    
#     print('Merged PLACES file was not found, merging and saving...')
#     gr_placematch.roads2places(trailname,data_roads,points_per_batch_places, delta_places, buffersize, tol_area)
#     data_places = gr_utils.merge_places(trailname, data_roads, points_per_batch_places) # Merge the different sections
#     gr_utils.write_places(trailname, data_places) # Write the merged PLACES data
#     print('Saved.')
    
# else: # The merged PLACES file does exist
    
#     print('Loading merged PLACES file...')
#     data_places = gr_utils.read_places(trailname) # Read the merged PLACES file
#     print('Loaded.')

Merged PLACES file was not found, merging and saving...
Handling batch 0 of 41 that covers road segments 0 through 99...
There are 77 uncorrected landuse places
There are 2 corrected landuse places
   Writing outputs to file...
   Finished this batch.
Handling batch 1 of 41 that covers road segments 100 through 199...


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


There are 113 uncorrected landuse places
There are 3 corrected landuse places
   Writing outputs to file...
   Finished this batch.
Handling batch 2 of 41 that covers road segments 200 through 299...


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


There are 370 uncorrected landuse places


...

   Writing outputs to file...
   Finished this batch.
Handling batch 40 of 41 that covers road segments 4000 through 4099...
There are 473 uncorrected landuse places


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


There are 3 corrected landuse places
   Writing outputs to file...
   Finished this batch.
Handling batch 41 of 41 that covers road segments 4100 through 4168...
There are 173 uncorrected landuse places
There are 2 corrected landuse places
   Writing outputs to file...
   Finished this batch.


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


Saved.


## Evaluating PROCESSED properties

In [18]:
# filename_processed = 'data_output/' + trailname + '_processed.csv'

# if not os.path.isfile(filename_processed): # The PROCESSED file does not exist
    
#     print('The PROCESSED file was not found, processing and saving...')
#     data = gr_process.places2processed(data_places,tol_d,types_slow,types_heavy,tracktype_p0,tracktype_p1,tracktype_p2,surface_p0,surface_p1,highway_p1) # Determine traffic/development/paved status & GR types
#     gr_utils.write_processed(trailname, data) # Write the processed data
#     print('Saved.')
    
# else: # The PROCESSED file does exist
    
#     print('The PROCESSED file already exists, so nothing needs to be done.')

The PROCESSED file was not found, processing and saving...
Saved.
